## 3.3 线性回归的简洁实现

随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。在本节中，我们将介绍如何使用MXNet提供的Gluon接口更方便地实现线性回归的训练。

### 3.3.1 生成数据集

我们生成与上一节中相同的数据集。其中features是训练数据特征，labels是标签。

In [2]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(loc=0, scale=1, shape=(num_examples, num_inputs)) # 均值为0，标准差为1
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(loc=0, scale=0.01, shape=labels.shape) # 均值为0，标准差为0.01

### 3.3.2 读取数据集

Gluon提供了data包来读取数据。由于data常用作变量名，我们将导入的data模块用添加了Gluon首字母的假名gdata代替。在每一次迭代中，我们将随机读取包含10个数据样本的小批量(一个epoch迭代100次)。

In [3]:
from mxnet.gluon import data as gdata

batch_size = 10
# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
# 随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

这里data_iter的使用与上一节中的一样。让我们读取并打印所有的(100个)小批量数据样本及对应的标签。

In [4]:
i = 0

for X, y in data_iter:
    i += 1
    print("第" + str(i) + "个小批量数据样本及对应的标签：")
    print(X, y)
    # break

第1个小批量数据样本及对应的标签：

[[-2.3526468   0.04862338]
 [ 0.05486475 -0.0292252 ]
 [ 0.6331259   0.2607249 ]
 [ 1.6035446   0.16749443]
 [-0.97968626  0.2906984 ]
 [ 0.21730663  0.22825038]
 [-0.81632394  0.7855458 ]
 [ 0.30383104 -0.80775887]
 [ 1.6067387   0.18408279]
 [-0.5459446  -2.3556297 ]]
<NDArray 10x2 @cpu(0)> 
[-0.660837    4.409067    4.5962906   6.828963    1.2627882   3.8540733
 -0.11465154  7.541996    6.7980385  11.099875  ]
<NDArray 10 @cpu(0)>
第2个小批量数据样本及对应的标签：

[[-0.47827247 -1.1739492 ]
 [-0.4847113   1.1821395 ]
 [-1.5869852  -0.47912028]
 [ 2.4302447   0.05513681]
 [-0.6376902   0.34791994]
 [-1.9574593   0.7722914 ]
 [ 0.74956447  1.3364263 ]
 [ 1.7608272  -1.0487508 ]
 [ 1.3810002  -0.76296663]
 [ 1.733211   -0.0860012 ]]
<NDArray 10x2 @cpu(0)> 
[ 7.2272377 -0.7929086  2.646202   8.877103   1.7478077 -2.317101
  1.1604966 11.291719   9.539722   7.9535317]
<NDArray 10 @cpu(0)>
第3个小批量数据样本及对应的标签：

[[-0.39261648 -0.06174479]
 [-0.9625808   1.0641294 ]
 [-0.8330337  -1.925278

### 3.3.3 定义模型

在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述模型是怎样计算的。当模型结构变得更复杂时，这些步骤将变得更繁琐。其实，Gluon提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用Gluon更简洁地定义线性回归。

首先，导入nn模块。实际上，“nn”是neural networks(神经网络)的缩写。顾名思义，该模块定义了大量神经网络的层。我们先定义一个模型变量net，它是一个Sequential实例。在Gluon中，Sequential实例可以看做是一个串联各个层的容器。在构造模型时，我们在该容器中依次添加层，容器中的每一层将依次计算并将输出作为下一层的输入。

In [5]:
from mxnet.gluon import nn

net = nn.Sequential()

In [6]:
net

Sequential(

)

回顾图3.1中线性回归在神经网络图中的表示。作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，线性回归的输出层又叫全连接层。在Gluon中，全连接层是一个Dense实例。我们定义该层输出个数为1。

In [7]:
net.add(nn.Dense(1))

In [8]:
net

Sequential(
  (0): Dense(None -> 1, linear)
)

值得一提的是，在Gluon中我们无须指定每一层输入的形状，例如线性回归的输入个数。当模型得到数据时，例如后面执行`net(X)`时，模型将自动推断出每一层的输入个数。我们将在之后“深度学习计算”一章详细介绍这种机制。Gluon的这一设计为模型开发带来便利。

### 3.3.4 初始化模型参数

在使用`net`前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。我们从MXNet导入`init`模块。该模块提供了模型参数初始化的各种方法。这里的`init`是`initializer`的缩写形式。我们通过`init.Normal(sigma=0.01)`指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。偏差参数默认会初始化为零。

In [9]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

### 3.3.5 定义损失函数

在Gluon中，`loss`模块定义了各种损失函数。我们用假名`gloss`代替导入的`loss`模块，并直接使用它提供的平方损失作为模型的损失函数。

In [10]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss() # 平方损失又称L2范数损失

### 3.3.6 定义优化算法

同样，我们也无须实现小批量随机梯度下降。在导入Gluon后，我们创建一个`Trainer`实例，并指定学习率为0.03的小批量随机梯度下降（`sgd`）为优化算法。该优化算法将用来迭代`net`实例所有通过`add`函数嵌套的层所包含的全部参数。这些参数可以通过`collect_params`函数获取。

In [11]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

### 3.3.7 训练模型

在使用Gluon训练模型时，我们通过调用`Trainer`实例的`step`函数来迭代模型参数。上一节中我们提到，由于变量`l`是长度为`batch_size`的一维`NDArray`，执行`l.backward()`等价于执行`l.sum().backward()`。按照小批量随机梯度下降的定义，我们在`step`函数中指明批量大小，从而对批量中样本梯度求平均。

In [12]:
num_epochs = 20

for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    total_loss = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, total_loss.mean().asnumpy()))

epoch 1, loss: 0.035073
epoch 2, loss: 0.000124
epoch 3, loss: 0.000048
epoch 4, loss: 0.000048
epoch 5, loss: 0.000048
epoch 6, loss: 0.000048
epoch 7, loss: 0.000048
epoch 8, loss: 0.000048
epoch 9, loss: 0.000048
epoch 10, loss: 0.000048
epoch 11, loss: 0.000048
epoch 12, loss: 0.000048
epoch 13, loss: 0.000048
epoch 14, loss: 0.000048
epoch 15, loss: 0.000048
epoch 16, loss: 0.000048
epoch 17, loss: 0.000048
epoch 18, loss: 0.000048
epoch 19, loss: 0.000048
epoch 20, loss: 0.000049


下面我们分别比较学到的模型参数和真实的模型参数。我们从`net`获得需要的层，并访问其权重（`weight`）和偏差（`bias`）。学到的参数和真实的参数很接近。

In [13]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4],
 
 [[ 2.0006096 -3.4001722]]
 <NDArray 1x2 @cpu(0)>)

In [14]:
true_b, dense.bias.data()

(4.2,
 
 [4.2006893]
 <NDArray 1 @cpu(0)>)

### 小结
* 使⽤Gluon可以更简洁地实现模型。
* 在Gluon中，data模块提供了有关数据处理的⼯具，nn模块定义了⼤量神经⽹络的层，loss模块定义了各种损失函数。
* MXNet的initializer模块提供了模型参数初始化的各种⽅法。

### 练习
* 如果将`l = loss(net(X), y)`替换成`l = loss(net(X), y).mean()`，我们需要将`trainer.step(batch_size)`相应地改成`trainer.step(1)`。这是为什么呢？

trainer.step(batch_size)中这个batch_size相当于把计算出来的梯度除以batch_size，因为l.backward()相当于nd.sum(l).bacward()，也就是把一个batch的loss都加起来求的梯度，所以通过除以batch_size能够弱化batch_size在更新参数时候的影响。我们在step函数中指明批量大小，从而对批量中样本梯度求平均。当把`l = loss(net(X), y)`替换成`l = loss(net(X), y).mean()`时，相当于在计算batch_size个样本的loss时，已经除以batch_size了，所以在l.backward()后进行SGD时，不需要除以batch_size了，除以1就行，即trainer.step(1)。

In [15]:
num_epochs = 10

for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y).mean()
        l.backward()
        trainer.step(1)
    total_loss = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, total_loss.mean().asnumpy()))

epoch 1, loss: 0.000048
epoch 2, loss: 0.000048
epoch 3, loss: 0.000048
epoch 4, loss: 0.000049
epoch 5, loss: 0.000048
epoch 6, loss: 0.000048
epoch 7, loss: 0.000048
epoch 8, loss: 0.000048
epoch 9, loss: 0.000048
epoch 10, loss: 0.000048


* 查阅MXNet文档，看看`gluon.loss`和`init`模块里提供了哪些损失函数和初始化方法。

#### gluon.loss模块里提供的损失函数有：
* https://mxnet.apache.org/versions/1.7.0/api/python/docs/api/gluon/loss/index.html#

#### initializer模块里提供的初始化方法有：
* https://mxnet.apache.org/versions/1.7.0/api/python/docs/api/initializer/index.html

* 如何访问`dense.weight`的梯度？

In [16]:
dense.weight.grad()


[[ 0.00407374 -0.00095672]]
<NDArray 1x2 @cpu(0)>